## Implementation Workflow

##### ■ Step 1: Canonicalization of the MPO (Damiano)

- Understand the canonical forms for MPOs as presented in the paper of Parker et al.
- Design a powerpoint slide with the definitions and corresponding graphical notations.
- Explain the algorithm needed how to bring a generic MPO into the respective forms.
- Implement a function which brings an MPO into specific two-site canonical form as needed in our algorithm.

In [ ]:
"""
    site_canonicalize(mpo::Vector, center::Int)

Algorithm bringing a given MPO into the two-site canonical form with orthogonality center at sites (center, center+1).

Input:
- `mpo::Vector`: List of the local tensors of MPO to be canonicalized
- `center::Int`: Index of the orthogonality center located at site (center, center+1)

Output:
- `mpo_canonical::Vector`: List of canonicalized local tensors of the input MPO
"""
function site_canonicalize(mpo::Vector, center::Int)

end

##### ■ Step 2: Efficient implementation of the sweeps (Matthias)

- The `XTRG_step` function should update $\rho(\beta)$ to $\rho(2 \beta)$ if possible and otherwise throw an error which aborts the simulation.
- After a number of `Nsweeps` iterations, the overlap $||\rho(2\beta)- \rho_{\text{opt}}(2\beta)||_F^2$ is computed to check whether the optimization has sufficiently converged as specified by the `tolerance` value. 
- Compute the inner product of the $\rho(2\beta)_{\text{diff}}$ operator while carefully performing the correct trace. The function should print a message to track that the result is still compatible with the `tolerance`. In this way, be can later adjust the number of `Nsweeps` that is appropriate for our simulation purposes.
- At the beginning of the step, we left-canonicalize with `leftcanonicalmpo` from MPO.jl
- As we work with canonicalized MPOs, the result of contraction in Fig. 17a of Appendix D in the main paper is exactly the optimal two-site tensor. Applying the SVD on this tensor, we easily obtain the new tensors $C_i$ and $C_{i+1}$ to update our state $\rho(2\beta)$.
- To start the sweeping it may be advantageous to compute all the left environments in advance. (Damiano agrees!) In the process of sweeping from right to left, we may then delete one left environment in the environment array `Envs` and replace it with the new right environment step by step. This should be similar to the data structure in the 2-site DMRG algorithm.  There, in particular, `Hlr[l + 1]` contains the left/right environment tensor relative to site `l` when the algorithm/sweeping is working at a couple of site that are on the right/left of site `l`, respectively (watch the indices!)
- Following the 2-site DMRG algorithm, i.e. the implementation/indexing/... of `DMRG_2site`, is a good idea also for implementing the sweeps. There, the sweeping works like this: <br>
-- Start a for cycle with the index `itS = (1:Nsweeps)` <br>
-- RIGHT TO LEFT SWEEP: `for itL = (L:-1:2)`, update as in appendix D of main ref. `mpo2[itL-1]` and `mpo2[itL]`. This involves `Vlr[itL-1]` as left environment $V_L$ and `Vlr[itL + 2]` as right environment $V_R$. It also involves an SVD to get `mpo2[itL-1]` and `mpo2[itL]` out of the 2-site optimized tensor ($C_i C_{i+1}$ in the paper). In particular, the updated `mpo2[itL]` must be an "MPO-right isometry", so that the updated `mpo2`  will be "MPO-site canonical" with new isometry center at site `itL - 1`. After that, update `Vlr[itL + 1]` using `updateLeft_mod` (and an index permutation since we are "updating leftwards")<br>
-- LEFT TO RIGHT SWEEP: similar to above: `for itL = (1:L-1)`, update `mpo2[itL]` and `mpo2[itL+1]` using `Vlr[itL] ` and `Vlr[itL + 3]`. Then update `Vlr[itL + 1]` using `updateLeft_mod`<br>
-- end of cycle with index itS
- Understand how contracting the SVD decomposed tensors preserves the site-canonical form of the MPO for subsequent updates.

In [ ]:
"""
    XTRG_step(rho::Vector, beta::Float64, mode::Bool, Nsweeps::Int, tolerance:Float64)

Function performing a single update of the XTRG algorithm from inverse temperatures beta ---> 2*beta

Input:
- `rho::Vector`: List of the (canonicalized) local tensors of the MPO corresponding to the quantum state at inverse temperature beta.
- `beta::Float64`: Current inverse temperature of the input state rho.
- `square::Bool`: If boolean is true initialize the updated rho as the square of rho
- `Nsweeps::Int`: Number of sweeps performed in the variational DMRG-type optimization along one direction of the chain.
- `tolerance:Float64`: Threshold value for which the locally optimized tensor rho_new is assumed converged.

Output:
- `rho_new::Vector`: List of (canonicalized) local tensors of the MPO corresponding to the quantum state at inverse temperature 2*beta.
- `beta::Float64`: Increased inverse temperature 2*beta for the state rho_new. 
"""
function XTRG_step(rho::Vector, beta::Float64, square::Bool, Nsweeps::Int=5, tolerance::Float64=1e-10)

    beta += beta

    # Choose initialization mode
    if square == true
        rho_init = square_mpo(rho)
    else
        rho_init = rho

    rho2 = deepcopy(rho_init)
    # Here comes the variational DMRG-type sweeping using the above methods modifying rho_new in place
    # 
    # 
    # 
    # 

    # Evaluate whether the optimization has sufficiently converged
    rho2_diff = add_mpo(rho2, [-rho2_new[1], rho2_new[2:end]])

    return rho2_new, beta

end

##### ■ Step 3: Assembling the XTRG Algorithm

- This function should implement a dictionary associating the modeled inverse temperatures $\beta$ with the corresponding quantum states $\rho(\beta)$.
- It should coordinate the calls to the `XTRG_step` function and abort the simulation if the `XTRG_step` function does not return a new  state.

In [ ]:
"""
    XTRG_algorithm(L::Int, beta0::Float64, betamax::Float64)

Function executing the XTRG algorithm to simulate the XY-Hamiltonian for a spin-1/2 system over a given temperature range.

Input:
- `L::Int`: length of the one-dimensional spin-1/2 system.
- `beta0::Float64`: initial inverse temperature to start the XTRG algorithm.
- `betamax::Float64`: maximal inverse temperature at which the XTRG algorithm is stopped.

Output:
- `res::Dict{Float64, Vector{Array{ComplexF64, 4}}}`

"""
function XTRG_algorithm(beta0::Float64, betamax::Float64)

end